---
title: Hamiltonian Simulation
date: 2024-08-08
authors:
  - name: Gerardo Suarez
---

### When the Hamiltonian is Physical

When the Hamiltonian is physical we can take the same steps as people who 
simulate the Lindblad master equation. I followed the 
[Lin Lin paper](https://arxiv.org/pdf/2311.15533). Here's a bit of the paper
but instead of using SDE schemes, I do it on the ensemble. To
obtain the Krauss operators (I also only do it to first order, because it is
simplest and should work when $dt\to0$ but probably one should consider
higher orders). 

### Krauss Operator from a Lindbladian

:::{warning}
This might be a mistake from the very beginning as Pseudomodes is not a CPTP 
map.However numerically, I've never seen any issue with positivity when enough 
levels are considered in the modes. So from here I'm assumming it will be CPTP
:::

The logic here is to start from the master equation

\begin{equation}
\dot{\rho(t)} =\mathcal{L}(\rho(t))
\end{equation}

From the definition of derivative this means

\begin{equation}
\lim_{dt \to 0}\frac{\rho(t+dt)-\rho(t)}{dt} =\mathcal{L}(\rho(t))
\end{equation}

For now let us forget about the limit, but work our quantities approximately
and to order $\mathcal{O}(dt^{2})$ so that

\begin{equation}
\rho(t+dt) \approx \rho(t)+ \mathcal{L}(\rho(t)) dt + \mathcal{O}(dt^{2})
\end{equation}

Since the map is CPTP then it must have a sum operator representation (Krauss
representation) so

\begin{equation}
\rho(t+dt) = \sum_{k} M_{k} \rho(t) M_{k}^{\dagger}
\end{equation}

What's left now is to find what the krauss operators should be. This is known
and can be seen for example in 
[Lidar's lecture notes](https://arxiv.org/abs/1902.00967). Here I do some extra
algebra to illustrate how to generalize to higher order schemes

Since this is lowest order then, we propose the Krauss operators

\begin{align}
M_{0} = \mathcal{1} + A dt \\ 
M_{k} = \sqrt{dt} B_{k}
\end{align}

Then we find that 

:::{warning}
I did these calculations by hand, but I am really lazy when it comes to latexing
so I decided to use sympy for intermediate steps, if there's any inconsistency
I can just latex those steps 😸
:::

In [62]:
from sympy import *
from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum import Commutator,AntiCommutator
from sympy.physics.quantum import TensorProduct
A=symbols("A",commutative=False)
rho=symbols("rho",commutative=False,cls=Function)
B,B1,Bk = symbols("B B_{1} B_{k}", commutative=False)
dt,t,N,k=symbols("dt t N k",real=True,positive=True)

In [2]:
M0=1+A*dt
Mk=sqrt(dt)*Bk

In [3]:
sumrepre = M0*rho(t)*Dagger(M0)+Sum(Mk*rho(t)*Dagger(Mk), (k, 1, N))
sumrepre=sumrepre.expand()+O(dt**2)
Eq(rho(t+dt),sumrepre)

Eq(rho(dt + t), Sum(dt*B_{k}*rho(t)*Dagger(B_{k}), (k, 1, N)) + rho(t) + dt*rho(t)*Dagger(A) + dt*A*rho(t) + O(dt**2))

Notice the series of Krauss operators we used have identical contributions (in
their form, and could be represented as a sum). Notice on the other hand we have

\begin{equation}
\rho(t+dt) \approx \rho(t)+ \mathcal{L}(\rho(t)) dt + \mathcal{O}(dt^{2})
\end{equation}

By replacing the lindbladian one obtains

\begin{equation}
\rho(t+dt) \approx \rho(t)+ \left( -i[H,\rho(t)] + \sum_{k} L_{k} \rho(t) 
L_{k}^{\dagger} - \frac{\{L_{k}^{\dagger} L_{k} , \rho(t)\}}{2}\right) dt 
\end{equation}

where the $L_{k}$ are the jump operators. Then by comparison we can find the required Krauss operators, First let us note
That to generate a commutator A must be an Anti-Hermitian matrix, and to 
generate an anticommutator A must be Hermitian. If we choose A to be a sum of a
Hermitian and Anti-hermitian matrix then



In [4]:
H,K = symbols("H K", commutative=False)
Eq(A,-I*H+K)

Eq(A, -I*H + K)

In [5]:
sumrepre=sumrepre.subs(A, -I*H+K).subs(Dagger(K), K).subs(Dagger(H), H).expand()
Eq(rho(t+dt), sumrepre)

Eq(rho(dt + t), Sum(dt*B_{k}*rho(t)*Dagger(B_{k}), (k, 1, N)) + rho(t) + dt*rho(t)*K + I*dt*rho(t)*H + dt*K*rho(t) - I*dt*H*rho(t) + O(dt**2))

Which we can simplify to



In [6]:
sumrepre = sumrepre.collect(dt).subs(I*rho(t)*H-I*H*rho(t), -I*Commutator(H,rho(t)))
sumrepre = sumrepre.subs(rho(t)*K+K*rho(t), AntiCommutator(K, rho(t)))
Eq(rho(t+dt), sumrepre)

Eq(rho(dt + t), Sum(dt*B_{k}*rho(t)*Dagger(B_{k}), (k, 1, N)) + rho(t) + dt*({K,rho(t)} - I*[H,rho(t)]) + O(dt**2))

At this point notice that if we select the $B_k$ to be the jump operators
and K to be the corresponding anticommutator term

In [7]:
Lk = IndexedBase("L", commutative=False)
sumrepre = sumrepre.subs(Bk, Lk).subs(K,- Sum(Dagger(Lk[k])*Lk[k], (k, 1, N))/2)
Eq(rho(t+dt), sumrepre)

Eq(rho(dt + t), Sum(dt*L*rho(t)*Dagger(L), (k, 1, N)) + rho(t) + dt*(-{Sum(Dagger(L[k])*L[k], (k, 1, N)),rho(t)}/2 - I*[H,rho(t)]) + O(dt**2))

We have obtained the First order scheme to obtain the Linblad master equation
Krauss operators. To obtain Higher order schemes One can notice that the solution
to the master equation is 

\begin{equation}
\rho(t) = e^{\mathcal{L}t} \rho(0)
\end{equation}

and

\begin{equation}
\rho(t+dt) = e^{\mathcal{L}(t+dt)} \rho(0) =e^{\mathcal{L}dt} \rho(t) 
\end{equation}

And then expand the series of the exponential. Similarly one should increase the
order of the krauss operator guess  by one and find the appropiate operators

### Notice We could have not guess K and use the Completeness relation of krauss operators

In this case it was not needed but it might be useful to find relations in higher order
schemes and to check the krauss operators are ok. So next we find K this way

The completeness relation indicates

\begin{align}
\sum_{k} M_{k}^{\dagger} M_{k} = \mathcal{1}
\end{align}

Since in our schemes we are numerically approximating to $\mathcal{O}(dt^{2})$
then 

\begin{align}
\sum_{k} M_{k}^{\dagger} M_{k} = \mathcal{1} +\mathcal{O}(dt^{2})
\end{align}

In [8]:
M0c = M0.subs(A,-I*H+K)
cpt=Dagger(M0c)*M0c+dt* Sum(Dagger(Lk[k])*Lk[k], (k, 1, N))
eqs=Eq(1,cpt.expand().subs(Dagger(H), H).subs(Dagger(K),K)+O(dt**2))
eqs

Eq(1, 1 + dt*Sum(Dagger(L[k])*L[k], (k, 1, N)) + 2*dt*K + O(dt**2))

Which we can solve to find

In [10]:
solve(eqs,K)[0]

-Sum(Dagger(L[k])*L[k], (k, 1, N))/2 + O(dt)

Now that we have found the Krauss operators one may simply ask if one can 
follow the same scheme to obtain the Krauss operators of a pseudomode equation.
Since I have not seen positivity issues I do think it's possible. but the naive 
approach to it yields and inconsistency 

### Same derivation with a non-Hermitian Hamiltonian


Any non-Hermitian matrix can be split into the sum of a Hermitian and Anti-Hermitian
Matrix such that I can write the unphysical Hamiltonian H as 

$$H=H_{0} + i H_{u}$$

Then the Lindblad equation turns into

\begin{equation}
\rho(t+dt) \approx \rho(t)+ \left( -i[H_{0},\rho(t)] + [H_{u},\rho(t)] + \sum_{k} L_{k} \rho(t) 
L_{k}^{\dagger} - \frac{\{L_{k}^{\dagger} L_{k} , \rho(t)\}}{2}\right) dt 
\end{equation}

Following the same strategy as before only A changes (the part that generated 
the commutator), so the change is only on $M_{0}$ We neglect the part that 
contains K as that one does not change

In [17]:
ans=(M0*rho(t)*Dagger(M0)).subs(A,-I*H).expand()+O(dt**2)
ans

rho(t) + I*dt*rho(t)*Dagger(H) - I*dt*H*rho(t) + O(dt**2)

Substitute 
$$H=H_{0} + i H_{u}$$

In [19]:
H0,Hu=symbols("H_0 H_u",commutative=False)

In [32]:
ans=ans.subs(H, H0+I*Hu).expand().subs(Dagger(H0),H0).subs(Dagger(Hu),Hu)
ans=ans.collect(dt).subs(I*rho(t)*H0-I*H0*rho(t), -I*Commutator(H0, rho(t)))
ans.subs(rho(t)*Hu+Hu*rho(t), AntiCommutator(Hu, rho(t)))

rho(t) + dt*({H_u,rho(t)} - I*[H_0,rho(t)]) + O(dt**2)

By comparison we would need

$$\{H_{u},\rho(t)\} =[H_{u},\rho(t)]$$

Which cannot be satisfied

Even though this calculation was a failure. 
Perhaps one would need to  use higher orders, or reorder the terms in another 
Fashion. I do believe it would be easier if I don't use the general formulation
but the jump operators as $a$ and $a^{\dagger}$ and the unphysical part of the 
Hamiltonian to have the form $\sum_{k} \omega_{k} a_{k}^{\dagger}a_{k}$ where $\omega_{k}$ is
complex.


While I look into it. Asume the Hamiltonian is Physical and then extrapolation
is done. One has several schemes to simulate Krauss operators in a quantum 
circuit simulator. Let us go with the scheme in the [Lin Lin paper](https://arxiv.org/pdf/2311.15533)

Even though in their case is not so bad, Here I illustrate why I don't like this
Hamiltonian approach in the first order. Perhaps it is better to use the other
[2nd Lin Lin paper](https://arxiv.org/pdf/2308.15676v4) though I think
the number of  ancillas needed will be bigger. I also need to try the 
[dilation paper](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.91.032113)


### Hamiltonian Simulation From Krauss Operators

#### In this section all traces are partial traces with respect to the ancilla

The paper suggests using  the Stinespring representation of the Krauss Operators
Namely. Finding an ancilla and a matrix $\mu$ such that

\begin{equation}
Tr(\mu)=\sum_{k} M_{k} \rho(t) M_{k}^{\dagger}
\end{equation}

To achieve this we can use the matrix $\mu$

\begin{equation}
\mu = \begin{pmatrix}
M_{0} \rho(t) M_{0}^{\dagger} & M_{0} \rho(t) M_{1}^{\dagger} 
&\dots &M_{0} \rho(t) M_{k}^{\dagger}  \\
M_{1} \rho(t) M_{0}^{\dagger} & M_{1} \rho(t) M_{1}^{\dagger}
& \dots &M_{1} \rho(t) M_{k}^{\dagger}  \\
\vdots &\vdots& \ddots &\vdots  \\
M_{k} \rho(t) M_{0}^{\dagger} &\dots &\dots  &M_{k} \rho(t) M_{k}^{\dagger} 
\end{pmatrix}
\end{equation}

Which can be easily constructed by requiring one ancilla qubit for each jump
operator. We consider all the ancillas to be on the ground state such 
that the state of the ancillas is

$\ket{0}\bra{0} = \begin{pmatrix}1 &0&\dots&0 \\
0&0&\dots&0\\
\vdots &\vdots &\vdots &\vdots \\
0&0&\dots&0 \end{pmatrix}$


Then one can obtain $\mu$ by 

\begin{equation}
\mu=A \ket{0}\bra{0} \otimes \rho  A^{\dagger}
\end{equation}

Where 

$$ A= \begin{pmatrix} M_{0} & M_{1}^{\dagger} &\dots & M_{k}^{\dagger}
\\ M_{1} & 0 &\dots & 0 \\ \vdots & 0 &\dots & 0 \\M_{k} & 0 &\dots & 0 \\
\end{pmatrix} $$

Then one can express the operator sum representation as 

$$Tr(A \ket{0}\bra{0} \otimes \rho  A^{\dagger})=\sum_{k} M_{k} \rho(t) 
M_{k}^{\dagger}$$

To have a Hamiltonian simulation of the Krauss representation we want to find
a unitary such that 
$$Tr(U \ket{0}\bra{0} \otimes \rho  U^{\dagger}) 
= Tr(A \ket{0}\bra{0} \otimes \rho  A^{\dagger})=\sum_{k} M_{k} \rho(t) 
M_{k}^{\dagger}$$

Or at least to order $\mathcal{O}(dt^{2})$. One of the insights of the paper is
to write U as

$$U=e^{-i \sqrt{dt} \bar{H}}$$

Where 

$$\bar{H}= \begin{pmatrix}H_{0} & H_{1}^{\dagger} &\dots & H_{k}^{\dagger}
\\ H_{1} & 0 &\dots & 0 \\ \vdots & 0 &\dots & 0 \\H_{k} & 0 &\dots & 0 \
 \end{pmatrix}$$

 with Hermitian $H_{0}$. Then One can obtain $\bar{H}$ from taylor expanding the
 exponential and matching the same order terms. For simplicity here I do it
 for 4 jump operators

In [112]:
Hams=IndexedBase("H", commutative=False)
M = IndexedBase("M", commutative=False)

def ladder(i,j,Hams,inverted=False):
    if (i==0)&(j==0):
        if inverted:
            return Dagger(Hams[j])
        else:
            return Hams[j]
    elif (i== 0)&(j!=0):
        return Dagger(Hams[j])
    elif j==0:
        return Hams[i]
    else:
        return 0
def hdilated(n,hams,inverted=False):
    return Matrix(n,n,lambda i,j: ladder(i,j,hams,inverted))

In [113]:
hdil=hdilated(5,Hams)
hdil

Matrix([
[H[0], Dagger(H[1]), Dagger(H[2]), Dagger(H[3]), Dagger(H[4])],
[H[1],            0,            0,            0,            0],
[H[2],            0,            0,            0,            0],
[H[3],            0,            0,            0,            0],
[H[4],            0,            0,            0,            0]])

In [114]:
Mdil = hdilated(5, M)
Mdil

Matrix([
[M[0], Dagger(M[1]), Dagger(M[2]), Dagger(M[3]), Dagger(M[4])],
[M[1],            0,            0,            0,            0],
[M[2],            0,            0,            0,            0],
[M[3],            0,            0,            0,            0],
[M[4],            0,            0,            0,            0]])

In [115]:
Mdild = hdilated(5, M,True)
Mdild

Matrix([
[Dagger(M[0]), Dagger(M[1]), Dagger(M[2]), Dagger(M[3]), Dagger(M[4])],
[        M[1],            0,            0,            0,            0],
[        M[2],            0,            0,            0,            0],
[        M[3],            0,            0,            0,            0],
[        M[4],            0,            0,            0,            0]])

In [116]:
def rhodilated(n):
    zero=Matrix([[1]+[0]*n])
    return zero.T*zero*rho(t)

In [118]:
rhod=rhodilated(4)
rhod

Matrix([
[rho(t), 0, 0, 0, 0],
[     0, 0, 0, 0, 0],
[     0, 0, 0, 0, 0],
[     0, 0, 0, 0, 0],
[     0, 0, 0, 0, 0]])

We check that this reproduces $\mu$

In [121]:
mu=Mdil*rhod*Mdild
mu

Matrix([
[M[0]*rho(t)*Dagger(M[0]), M[0]*rho(t)*Dagger(M[1]), M[0]*rho(t)*Dagger(M[2]), M[0]*rho(t)*Dagger(M[3]), M[0]*rho(t)*Dagger(M[4])],
[M[1]*rho(t)*Dagger(M[0]), M[1]*rho(t)*Dagger(M[1]), M[1]*rho(t)*Dagger(M[2]), M[1]*rho(t)*Dagger(M[3]), M[1]*rho(t)*Dagger(M[4])],
[M[2]*rho(t)*Dagger(M[0]), M[2]*rho(t)*Dagger(M[1]), M[2]*rho(t)*Dagger(M[2]), M[2]*rho(t)*Dagger(M[3]), M[2]*rho(t)*Dagger(M[4])],
[M[3]*rho(t)*Dagger(M[0]), M[3]*rho(t)*Dagger(M[1]), M[3]*rho(t)*Dagger(M[2]), M[3]*rho(t)*Dagger(M[3]), M[3]*rho(t)*Dagger(M[4])],
[M[4]*rho(t)*Dagger(M[0]), M[4]*rho(t)*Dagger(M[1]), M[4]*rho(t)*Dagger(M[2]), M[4]*rho(t)*Dagger(M[3]), M[4]*rho(t)*Dagger(M[4])]])

In [123]:
mu.trace()

M[0]*rho(t)*Dagger(M[0]) + M[1]*rho(t)*Dagger(M[1]) + M[2]*rho(t)*Dagger(M[2]) + M[3]*rho(t)*Dagger(M[3]) + M[4]*rho(t)*Dagger(M[4])

While at first order in the exponential we have

$$exp(x)\approx 1+x$$

In [129]:
U=eye(5)-I*sqrt(dt)*hdil
Ud = eye(5)+I*sqrt(dt)*hdil
U

Matrix([
[-I*sqrt(dt)*H[0] + 1, -I*sqrt(dt)*Dagger(H[1]), -I*sqrt(dt)*Dagger(H[2]), -I*sqrt(dt)*Dagger(H[3]), -I*sqrt(dt)*Dagger(H[4])],
[    -I*sqrt(dt)*H[1],                        1,                        0,                        0,                        0],
[    -I*sqrt(dt)*H[2],                        0,                        1,                        0,                        0],
[    -I*sqrt(dt)*H[3],                        0,                        0,                        1,                        0],
[    -I*sqrt(dt)*H[4],                        0,                        0,                        0,                        1]])

Again the goal is to have this emulate the krauss operators, which emulate the master equation

In [138]:
Eq(rho(t+dt), sumrepre)

Eq(rho(dt + t), Sum(dt*L*rho(t)*Dagger(L), (k, 1, N)) + rho(t) + dt*(-{Sum(Dagger(L[k])*L[k], (k, 1, N)),rho(t)}/2 - I*[H,rho(t)]) + O(dt**2))

By using

In [140]:
ansdil=(U*rhod*Ud).trace().expand().collect(sqrt(dt))
ansdil

sqrt(dt)*(I*rho(t)*H[0] - I*H[0]*rho(t)) + dt*(H[0]*rho(t)*H[0] + H[1]*rho(t)*Dagger(H[1]) + H[2]*rho(t)*Dagger(H[2]) + H[3]*rho(t)*Dagger(H[3]) + H[4]*rho(t)*Dagger(H[4])) + rho(t)

We further simiplify it to be

In [158]:
ansdil=ansdil.subs(I*rho(t)*hdil[0, 0]-I*hdil[0, 0]*rho(t), -I*Commutator(hdil[0, 0], rho(t))).subs(sum([hdil[i, 0]*rho(t)*hdil[0, i] for i in range(1,5)]),Sum(Hams[k]*rho(t)*Hams[k],(k,1,4)))
ansdil

-I*sqrt(dt)*[H[0],rho(t)] + dt*(H[0]*rho(t)*H[0] + Sum(H[k]*rho(t)*H[k], (k, 1, 4))) + rho(t)

To approximate the master equation notice that we can have 

$H_{0}=\sqrt(dt)H$

$H_{k}=L_{k}$

Which is the first order in the [Lin Lin paper](https://arxiv.org/pdf/2311.15533)
and results in 

In [161]:
ansdil.subs(Hams[0],sqrt(dt)*H).subs(Hams,Lk)

dt*(dt*H*rho(t)*H + Sum(L[k]*rho(t)*L[k], (k, 1, 4))) - I*dt*[H,rho(t)] + rho(t)

Them neglecting higher order terms one has

In [163]:
ansdil.subs(Hams[0], sqrt(dt)*H).expand().subs(Hams,Lk)+O(dt**2)

rho(t) + dt*Sum(L[k]*rho(t)*L[k], (k, 1, 4)) - I*dt*[H,rho(t)] + O(dt**2)

But this neglects the anticommutator bit 😭This is what I don't like but it can
be fixed by higher orders does not seem to affect accuracy too much

:::{note} TO DO! 👈
- [ ] Use higher order Schemes, and try to have a non-Hermitian Hamiltonian
- [ ] Test pseudomodes being CPTP by calculating 
$\tau:=(\mathcal{L} \otimes \mathrm{id}_d)(|\Omega\rangle\langle \Omega|)$ 
where $\ket{\Omega}$ is the maximally entangled state (if CPTP $\tau \geq 0$)
- [ ] Check other simulation schemes like the other lin lin paper or Clover's 
paper
- [ ] Actually do the Hamiltonian simulation, The ancillas need to be reset 
every timestep but this suceeds with probabulity one  in theory
:::